In [2]:
%pip install pyspark

  Using cached pyspark-3.5.0-py2.py3-none-any.whl
  Using cached py4j-0.10.9.7-py2.py3-none-any.whl (200 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder \
    .appName("Spark Profiling") \
    .getOrCreate()

# Assuming df is your DataFrame
# For demonstration, let's create a sample DataFrame
data = [(1, 2, None), (None, 5, 6), (7, None, None), (4, None, 10)]
columns = ["A", "B", "C"]
df = spark.createDataFrame(data, columns)

# Display summary statistics
df.describe().show()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/20 20:30:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+-------+---+------------------+------------------+
|summary|  A|                 B|                 C|
+-------+---+------------------+------------------+
|  count|  3|                 2|                 2|
|   mean|4.0|               3.5|               8.0|
| stddev|3.0|2.1213203435596424|2.8284271247461903|
|    min|  1|                 2|                 6|
|    max|  7|                 5|                10|
+-------+---+------------------+------------------+



In [7]:
from pyspark.sql.functions import col, count, when, lit
from pyspark.sql.functions import round as spark_round
from pyspark.sql import functions as F

def describe_with_null_counts(df):
    # Calculate count, mean, stddev, min, max
    describe_df = df.describe()

    # Calculate null counts for each column
    null_counts = df.select([F.sum(F.col(c).isNull().cast("int")).alias(c) for c in df.columns])

    # Convert null counts row to a DataFrame with the same schema as describe_df
    null_counts_row = spark.createDataFrame([null_counts.collect()[0]], schema=describe_df.schema)

    # Concatenate describe_df with the null counts row
    describe_df = describe_df.union(null_counts_row)

    return describe_df

# Usage example:
describe_with_null_counts(df).show()

PySparkValueError: [LENGTH_SHOULD_BE_THE_SAME] obj and fields should be of the same length, got 3 and 4.